In [ ]:
import collections
import numpy as np
import matplotlib.pyplot as plt
from functools import lru_cache
from itertools import product
from scipy.optimize import minimize

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate, XXPlusYYGate
from qiskit_aer.primitives import EstimatorV2 as Estimator
from qiskit_aer.primitives import SamplerV2 as Sampler


# --- Problem Definition & Helper Functions (Unchanged) ---

cubes_default_orientation = [
    {"Front": "G", "Back": "W", "Top": "R", "Bottom": "B", "Left": "R", "Right": "R"}, # Cube 1
    {"Front": "W", "Back": "B", "Top": "W", "Bottom": "G", "Left": "R", "Right": "B"}, # Cube 2
    {"Front": "R", "Back": "G", "Top": "B", "Bottom": "W", "Left": "R", "Right": "W"}, # Cube 3
    {"Front": "R", "Back": "G", "Top": "B", "Bottom": "B", "Left": "W", "Right": "G"}  # Cube 4
]

@lru_cache(maxsize=None)
def get_orientation_from_binary_string(cube_idx, b_str):
    initial_orientation = cubes_default_orientation[cube_idx]
    b0, b1, b2 = [int(b) for b in b_str]
    current_orientation = initial_orientation.copy()
    if b0 == 1:
        original = current_orientation.copy()
        current_orientation['Top'], current_orientation['Right'], current_orientation['Bottom'], current_orientation['Left'] = \
            original['Left'], original['Top'], original['Right'], original['Bottom']
    if b1 == 1:
        original = current_orientation.copy()
        current_orientation['Top'], current_orientation['Front'], current_orientation['Bottom'], current_orientation['Back'] = \
            original['Back'], original['Top'], original['Front'], original['Bottom']
    if b2 == 1:
        original = current_orientation.copy()
        current_orientation['Top'], current_orientation['Right'], current_orientation['Bottom'], current_orientation['Left'] = \
            original['Left'], original['Top'], original['Right'], original['Bottom']
    return current_orientation

def calculate_classical_cost(bitstring):
    current_cost = 0
    orientations = [get_orientation_from_binary_string(i, bitstring[3*i:3*i+3]) for i in range(4)]
    tb_counts = collections.defaultdict(int)
    lr_counts = collections.defaultdict(int)
    for o in orientations:
        tb_counts[o['Top']] += 1; tb_counts[o['Bottom']] += 1
        lr_counts[o['Left']] += 1; lr_counts[o['Right']] += 1
    for c in ["R", "B", "W", "G"]:
        current_cost += (tb_counts[c] - 2)**2
        current_cost += (lr_counts[c] - 2)**2
    return current_cost

NUM_QUBITS = 12

@lru_cache(maxsize=None)
def get_binary_poly_coeffs(cube_idx, face, color):
    y = np.zeros(8)
    for j, b_vars in enumerate(product([0, 1], repeat=3)):
        b_str = "".join(map(str, b_vars))
        orientation = get_orientation_from_binary_string(cube_idx, b_str)
        if orientation[face] == color:
            y[j] = 1
    M = np.zeros((8, 8))
    for i, z_vars in enumerate(product([0, 1], repeat=3)):
        z0, z1, z2 = z_vars
        M[i, :] = [1, z0, z1, z2, z0*z1, z0*z2, z1*z2, z0*z1*z2]
    coeffs = np.linalg.solve(M, y)
    poly_map = {}
    term_map = [(), (0,), (1,), (2,), (0, 1), (0, 2), (1, 2), (0, 1, 2)]
    for i, c in enumerate(coeffs):
        if not np.isclose(c, 0):
            poly_map[term_map[i]] = c
    return poly_map

def term_to_pauli(term_indices, coeff, num_qubits):
    op = SparsePauliOp("I" * num_qubits, coeffs=[coeff])
    identity = SparsePauliOp.from_list([("I" * num_qubits, 1)])
    for i in term_indices:
        pauli_str = ["I"] * num_qubits
        pauli_str[num_qubits - 1 - i] = "Z"
        z_i_op = (identity - SparsePauliOp("".join(pauli_str))) * 0.5
        op = op @ z_i_op
    return op

def get_indicator_pauli(cube_idx, face, color, num_qubits):
    poly_coeffs = get_binary_poly_coeffs(cube_idx, face, color)
    total_op = SparsePauliOp("I" * num_qubits, coeffs=[0])
    for local_terms, coeff in poly_coeffs.items():
        global_indices = [t + 3 * cube_idx for t in local_terms]
        pauli_term = term_to_pauli(global_indices, coeff, num_qubits)
        total_op += pauli_term
    return total_op.simplify()

@lru_cache(maxsize=1)
def build_cost_hamiltonian(num_qubits):
    print("Building cost Hamiltonian from constraints...")
    colors = ["R", "B", "W", "G"]
    N_ops = {"TopBottom": {}, "LeftRight": {}}
    for c in colors:
        N_ops["TopBottom"][c] = SparsePauliOp("I" * num_qubits, coeffs=[0])
        N_ops["LeftRight"][c] = SparsePauliOp("I" * num_qubits, coeffs=[0])
    for i in range(4):
        for c in colors:
            N_ops["TopBottom"][c] += get_indicator_pauli(i, 'Top', c, num_qubits)
            N_ops["TopBottom"][c] += get_indicator_pauli(i, 'Bottom', c, num_qubits)
            N_ops["LeftRight"][c] += get_indicator_pauli(i, 'Left', c, num_qubits)
            N_ops["LeftRight"][c] += get_indicator_pauli(i, 'Right', c, num_qubits)
    cost_op = SparsePauliOp("I" * num_qubits, coeffs=[0])
    identity_op = SparsePauliOp("I" * num_qubits, coeffs=[1])
    for group in ["TopBottom", "LeftRight"]:
        for c in colors:
            N_c = N_ops[group][c].simplify()
            term_op = (N_c @ N_c) - (4 * N_c) + (4 * identity_op)
            cost_op += term_op
    print("Hamiltonian built and cached.")
    return cost_op.simplify()

def create_qaoa_circ(params, cost_hamiltonian, p, mixer_type='xy'):
    num_qubits = cost_hamiltonian.num_qubits
    betas = params[:p]
    gammas = params[p:]
    qc = QuantumCircuit(num_qubits)
    qc.h(range(num_qubits))
    for i in range(p):
        evolution_gate = PauliEvolutionGate(cost_hamiltonian, time=gammas[i])
        qc.compose(evolution_gate.definition, inplace=True)
        qc.barrier()
        if mixer_type == 'xy':
            for j in range(4):
                q_indices = [3 * j, 3 * j + 1, 3 * j + 2]
                xy_gate_01 = XXPlusYYGate(2 * betas[i])
                qc.compose(xy_gate_01.definition, [q_indices[0], q_indices[1]], inplace=True)
                xy_gate_12 = XXPlusYYGate(2 * betas[i])
                qc.compose(xy_gate_12.definition, [q_indices[1], q_indices[2]], inplace=True)
                xy_gate_20 = XXPlusYYGate(2 * betas[i])
                qc.compose(xy_gate_20.definition, [q_indices[2], q_indices[0]], inplace=True)
        else:
            qc.rx(2 * betas[i], range(num_qubits))
        qc.barrier()
    return qc

# --- MODIFIED FUNCTION ---
def display_top_states_info(counts, num_states=5):
    """
    Calculates costs and probabilities for the top N most probable states and prints the results.
    """
    # Changed "Counts" header to "Probability"
    print(f"{'Rank':<5} | {'Bitstring':^{NUM_QUBITS}} | {'Probability':<15} | {'Classical Cost'}")
    # Adjusted separator length for the new header
    print("-" * (33 + NUM_QUBITS))
    
    if not counts:
        print("No measurement results to display.")
        print("-" * (33 + NUM_QUBITS))
        return

    # Calculate the total number of shots to find the probability
    total_shots = sum(counts.values())
    if total_shots == 0:
        print("Total shots is zero, cannot calculate probabilities.")
        print("-" * (33 + NUM_QUBITS))
        return
        
    sorted_counts = sorted(counts.items(), key=lambda item: item[1], reverse=True)[:num_states]
    
    for i, (bitstring, count) in enumerate(sorted_counts):
        # Calculate the probability for the current state
        probability = count / total_shots
        cost = calculate_classical_cost(bitstring)
        # Print the probability formatted to 4 decimal places
        print(f"{i+1:<5} | {bitstring:<{NUM_QUBITS}} | {probability:<15.4f} | {cost}")
    print("-" * (33 + NUM_QUBITS))


def run_qaoa_for_depth(p, cost_hamiltonian, estimator, sampler):
    def objective_function(params):
        qc = create_qaoa_circ(params, cost_hamiltonian, p)
        pub = (qc, cost_hamiltonian)
        job = estimator.run([pub])
        result = job.result()
        return result[0].data.evs

    print(f"\n{'='*15} depth p={p} {'='*15}")
    initial_params = np.random.rand(2 * p) * np.pi
    res = minimize(objective_function, initial_params, method='COBYLA', options={'maxiter': 250, 'disp': False})

    final_qc = create_qaoa_circ(res.x, cost_hamiltonian, p)
    final_qc.measure_all()
    
    sampler_job = sampler.run([final_qc], shots=10000)
    sampler_result = sampler_job.result()
    counts = sampler_result[0].data.meas.get_counts()
    display_top_states_info(counts, num_states=5)
    
    if not counts:
        return 0, 0 

    total_shots = sum(counts.values())
    most_probable_bitstring = max(counts, key=counts.get)
    cost_of_top_state = calculate_classical_cost(most_probable_bitstring)
    prob_of_top_state = counts[most_probable_bitstring] / total_shots if total_shots > 0 else 0
    
    return cost_of_top_state, prob_of_top_state

def main(max_depth):
    estimator = Estimator()
    sampler = Sampler()
    cost_hamiltonian = build_cost_hamiltonian(NUM_QUBITS)
    depths = []
    costs_of_top_states = []
    probs_of_top_states = []
    p = 5

    while p <= max_depth:
        top_cost, top_prob = run_qaoa_for_depth(p, cost_hamiltonian, estimator, sampler)
        depths.append(p)
        costs_of_top_states.append(top_cost)
        probs_of_top_states.append(top_prob)
        p += 5
    
    if not depths:
        print("\nNo data was generated to plot. This might be due to a low max_depth.")
        return
        
    print("\nPlotting final results...")
    fig, ax1 = plt.subplots(figsize=(12, 7))
    plt.title('QAOA Performance: Cost and Probability vs. Depth', fontsize=16)
    color1 = 'tab:blue'
    ax1.set_xlabel('QAOA Depth (p)', fontsize=12)
    ax1.set_ylabel('Cost of Most Probable State', color=color1, fontsize=12)
    ax1.plot(depths, costs_of_top_states, marker='o', linestyle='-', color=color1, label='Cost')
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.set_xticks(depths)
    ax1.set_ylim(bottom=max(-1, min(costs_of_top_states) - 2), top=max(costs_of_top_states) + 2)
    plt.setp(ax1.get_xticklabels(), rotation=45, ha="right")
    ax2 = ax1.twinx()  
    color2 = 'tab:green'
    ax2.set_ylabel('Probability of Most Probable State', color=color2, fontsize=12)
    ax2.plot(depths, probs_of_top_states, marker='s', linestyle='--', color=color2, label='Probability')
    ax2.tick_params(axis='y', labelcolor=color2)
    ax2.set_ylim(0, max(probs_of_top_states) * 1.1 if probs_of_top_states else 1)
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper center')
    ax1.grid(True, which='both', linestyle='--', linewidth=0.5)
    fig.tight_layout()  
    plt.show()

if __name__ == "__main__":
    main(max_depth = 200)

Building cost Hamiltonian from constraints...
Hamiltonian built and cached.

=============== depth p=5 ===============
Rank  |  Bitstring   | Probability     | Classical Cost
---------------------------------------------
1     | 100110110110 | 0.0022          | 12
2     | 110001001110 | 0.0021          | 8
3     | 100010110111 | 0.0020          | 8
4     | 110001100001 | 0.0020          | 8
5     | 110100001110 | 0.0020          | 8
---------------------------------------------

=============== depth p=10 ===============
